# TOAST3 test script
First try at producing an observed map with `TOAST3`.
- The following scripts are mainly taken (and adapted) from https://github.com/simonsobs/pwg-scripts/tree/master/pwg-tds/sat-obs-matrix/2022-mode-loss-study. 
- The best script to study for SAT TOD sims & mapmaking is this one: https://github.com/simonsobs/sotodlib/tree/master/workflows.
- `TOAST3` is quite different from `TOAST2`! You can find the source files here: https://github.com/hpc4cmb/toast/tree/toast3/src/toast

## Preparation


#### Load sohpc stack
````bash
module add /global/common/software/sobs/$NERSC_HOST/default/modulefiles`
module load sohpc
````

#### Install `TOAST3` and dependencies
````bash
cd; git clone https://github.com/simonsobs/sotodlib; cd sotodlib; pip install .
git clone https://github.com/simonsobs/pwg-scripts`
pip install -y pixell --user
pip install -y so3g --user
pip install -y toast==3.0.0a15 --user
python -c 'import toast; print(toast.__file__)'
````
Copy the resulting toast3 directory (only the directory, not the file path).

#### Add `bashrc` environment
- Add the following snippet to your ~/.bashrc file:
````bash
load_toast() {
    module use /global/common/software/sobs/$NERSC_HOST/default/modulefiles
    module load sohpc
    module load openmpi
    dir=[toast3 directory]
    export PATH="${dir}/bin:${PATH}"
    pysite=$(python3 --version 2>&1 | awk "{print $2}" | sed -e "s#\(.*\)\.\(.*\)\..*#\1.\2#")
    export PYTHONPATH="${dir}/lib/python${pysite}/site-packages:${PYTHONPATH}"
}
````
- Save changes with `source ~/.bashrc`.
- Now you can always load this envirmonment with `load_toast`.

### Make virtual environment for jupyter kernel
- Add a file named `kernel-helper.sh` to `~/.local/share/jupyter/kernels/toast3`:
````bash
#!/bin/bash
module use /global/common/software/sobs/$NERSC_HOST/default/modulefiles
module load sohpc
module load openmpi
dir=[toast3 directory]
export PATH="${dir}/bin:${PATH}"
pysite=$(python3 --version 2>&1 | awk "{print $2}" | sed -e "s#\(.*\)\.\(.*\)\..*#\1.\2#")
export PYTHONPATH="${dir}/lib/python${pysite}/site-packages:${PYTHONPATH}"
exec "$@"
````
- Then, `chmod +ux` it.
- Add another file named `kernel.json` to the same directory:
````json
{
    "argv": [
        "{resource_dir}/kernel-helper.sh",
        "python",
        "-m",
        "ipykernel_launcher",
        "-f",
        "{connection_file}"
    ],
    "display_name": "TOAST3",
    "language": "python"
}
````
- The kernel `TOAST3` will now be available as a jupyter kernel on NERSC.
- Next, restart this notebook and load the `TOAST3` kernel.

In [8]:
import sys
import toast
import toast.scripts
print('This is your toast3 install path:\n', '/'.join(toast.scripts.__file__.split('/')[:-1]))

This is your toast3 install path:
 /global/homes/k/kwolz/.local/cmbenv-20220322/lib/python3.9/site-packages/toast/scripts


### Making an observing schedule
Replace the toast3 path in the next line with the previous line:

In [2]:
!$sys.executable /global/homes/k/kwolz/.local/cmbenv-20220322/lib/python3.9/site-packages/toast/scripts/toast_ground_schedule.py \
@schedule_sat.par \
@patches_sat.txt \
--elevations-deg 50,60 \
--elevation-change-limit-deg 1 --elevation-change-time 1800 \
--out schedules/schedule_sat.fixed_el.txt

TOAST INFO: Adding patch "south"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+000.000..+011.613"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+011.613..+023.226"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+023.226..+034.839"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+034.839..+046.452"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+046.452..+058.065"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+058.065..+069.677"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+069.677..+081.290"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+081.290..+092.903"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+301.935..+313.548"
TOAST INFO: Rectangular format
TOAST INFO: Adding patch "DEC-050..-030_RA+313.548..+325.161"
TOAST INFO: Rectangular format
TOAST 

### Writing the TOAST simulation & data reduction script

In [3]:
%%writefile run.sh
#!/bin/bash

export OMP_NUM_THREADS=1
sotodlibdir=~/sotodlib/workflows

logfile=toast.log

if [[ -e $logfile ]]; then
    echo "$logfile exists"
fi

outdir=toast_output
mkdir -p $outdir

echo "Writing $logfile"

TOAST_LOGLEVEL=debug mpirun -np 4 python \
$sotodlibdir/toast_so_sim.py \
 --config 2022.toml \
 --schedule schedules/schedule_sat.fixed_el.txt \
 --bands SAT_f150 \
 --telescope SAT1 \
 --thinfp 64 \
 --sample_rate 40 \
 --sim_noise.enable \
 --sim_atmosphere_coarse.disable \
 --sim_atmosphere.disable \
 --out $outdir \
 --job_group_size 1 \
 >& $logfile
exit

Overwriting run.sh


### Running the TOAST simulation & data reduction script
The following run contains some bugs, but it is (I think) a good starting point for integrating filtering in our MASTER pipeline.

In [4]:
import subprocess as sp
command = f"bash run.sh"

# if nersc_host is not None:
#     command = "sbatch toast_satellite_sim.slurm"
# else:
#     # Just use mpirun
#     command = "mpirun -np 4 toast_satellite_sim.py @toast_satellite_sim.par"

print(command, flush=True)
sp.check_call(command, stderr=sp.STDOUT, shell=True)

bash run.sh
toast.log exists
Writing toast.log


CalledProcessError: Command 'bash run.sh' returned non-zero exit status 1.